In [ ]:
import numpy as np
import pandas as pd

default_path = '../input/'

In [ ]:
train_df = pd.read_csv(default_path+'sales_train.csv')
items_df = pd.read_csv(default_path+'items.csv')
test_df = pd.read_csv(default_path+'test.csv')

print(train_df.shape, test_df.shape)

In [ ]:
train_df['date'] = pd.to_datetime(train_df['date'], format='%d.%m.%Y')

In [ ]:
train_df.head()

In [ ]:
dataset = train_df.pivot_table(index=['item_id', 'shop_id'],values=['item_cnt_day'], columns='date_block_num', fill_value=0)

In [ ]:
dataset = dataset.reset_index()
dataset.head()

In [ ]:
dataset = pd.merge(test_df, dataset, on=['item_id', 'shop_id'], how='left')
dataset = dataset.fillna(0)
dataset.head()

This gives us the required data to work on that is the item sold on each day

In [ ]:
dataset = dataset.drop(['shop_id', 'item_id', 'ID'], axis=1)
dataset.head()

In [ ]:
X_train = np.expand_dims(dataset.values[:, :-1], axis=2)
y_train = dataset.values[:, -1:]

X_test = np.expand_dims(dataset.values[:, 1:], axis=2)
print(X_train.shape, y_train.shape, X_test.shape)

defining **LSTM** layers and the imports

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

In [ ]:
model = Sequential()
model.add(LSTM(64, input_shape=(33, 1)))
model.add(Dropout(0.3))
model.add(Dense(1))

model.compile(loss='mse',
             optimizer='adam',
             metrics=['mean_squared_error'])

model.summary()

In [ ]:
model_fitting = model.fit(X_train, y_train, batch_size=4096, epochs=10)

In [ ]:
model_prediction = model.predict(X_test)
model_prediction = model_prediction.clip(0, 20)

In [ ]:
submission = pd.DataFrame({'ID': test_df['ID'], 'item_cnt_month': model_prediction.ravel()})
submission.to_csv('submission.csv',index=False)